In [1]:
# -*- coding:utf8 -*-
import os
import csv
import pandas as pd
import numpy as np
import math
import time
import random
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
path_train = 'train.csv'

In [3]:
train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')

In [4]:
train.describe()

,TERMINALNO,TIME,TRIP_ID,LONGITUDE,LATITUDE,DIRECTION,HEIGHT,SPEED,CALLSTATE,Y
count,69306.000000,6.930600e+04,69306.000000,69306.000000,69306.000000,69306.000000,69306.000000,69306.000000,69306.000000,69306.000000
mean,52.104233,1.478202e+09,38.330996,114.547988,32.039129,178.369405,172.926933,9.192463,1.635832,0.221681
std,29.131318,4.355266e+06,33.879349,6.484526,4.960202,105.717157,365.845400,8.779666,1.948844,1.650450
min,1.000000,1.467605e+09,1.000000,82.823676,21.574338,-1.000000,-271.488617,-1.000000,0.000000,0.000000
25%,28.000000,1.475554e+09,12.000000,109.156754,29.102076,88.000000,6.000000,1.520000,0.000000,0.000000
50%,54.000000,1.479284e+09,29.000000,114.539589,31.264475,178.000000,28.115479,7.150000,0.000000,0.000000
75%,78.000000,1.482121e+09,55.000000,120.184822,34.734129,267.000000,105.432938,14.011726,4.000000,0.000000
max,100.000000,1.483199e+09,223.000000,127.403597,44.000393,360.000000,3411.894775,53.480000,4.000000,20.742695


In [5]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

def conver_time(data):
    data['Conver_TIME'] = data.TIME.apply(timestamp_datetime)
#     data['month'] = data.Conver_TIME.apply(lambda x: int(x[5:7]))
    data['hour'] = data.Conver_TIME.apply(lambda x: int(x[11:13]))
#     data = data.drop('TIME',axis=1)
#     data = data.drop('Conver_TIME',axis=1)
    return data

def label_process(data):
    pre_label = data.drop_duplicates()
    return pre_label['Y']

In [6]:
def feature_process(data):
    set_data = set(data['TERMINALNO'])
    columns=['p_id',
             'maxTime','phonerisk','dir_risk','height_risk',
             'speed_max','speed_mean','speed_var','height_max','height_mean','height_var','sp_he_mean',
             'zao','wan','shenye',
             'weizhi_ratio','huchu_ratio','huru_ratio','liantong_ratio','duanlian_ratio'
            ]
    feature = pd.DataFrame(columns=columns)
    
    for p_id in set_data:
        tempData = data.loc[data['TERMINALNO'] == p_id]
        tempData = tempData.sort_values(["TIME"])

        tempTime = tempData["TIME"].iloc[0]
        tempSpeed = tempData["SPEED"].iloc[0]
        tempDir = tempData["DIRECTION"].iloc[0]
        tempHeight = tempData["HEIGHT"].iloc[0]

        maxTime = 0
        maxTimelist = []

        phonerisk = 0

        dir_risk = 0

        height_risk = 0
        zao=0
        wan=0
        shenye=0
        
        weizhi = 0
        huchu = 0
        huru = 0
        liantong = 0
        duanlian = 0

        for index, row in tempData.iterrows():
            
            hour = row['hour']
            if 7 <= hour <= 9:
                zao = 1
            elif 17 <= hour <= 19:
                wan = 1
            elif 0 <= hour < 7:
                shenye = 1

            if tempSpeed > 0 and row['CALLSTATE'] != 4:
                if row["CALLSTATE"] == 0:
                    phonerisk += math.exp(tempSpeed / 10) * 0.02
                else:
                    phonerisk += math.exp(tempSpeed / 10)
       
            if row["TIME"] - tempTime == 60:
                maxTime += 60
                tempTime = row["TIME"]

                dir_change = (min(abs(row["DIRECTION"] - tempDir), abs(360 + tempDir - row["DIRECTION"])) / 90.0)
                if tempSpeed != 0 and row["SPEED"] > 0:
                    dir_risk += math.pow((row["SPEED"] / 10), dir_change)

                height_risk += math.pow(abs(row["SPEED"] - tempSpeed) / 10,(abs(row["HEIGHT"] - tempHeight) / 100))
                
                tempHeight = row["HEIGHT"]

            elif row["TIME"] - tempTime > 60:
                maxTimelist.append(maxTime)
                maxTime = 0
                tempTime = row["TIME"]

                tempDir = row["DIRECTION"]
                tempHeight = row["HEIGHT"]
                tempSpeed = row["SPEED"]
                
            if row["CALLSTATE"] == 0:
                weizhi += 1
            elif row["CALLSTATE"] == 1:
                huchu += 1
            elif row["CALLSTATE"] == 2:
                huru += 1
            elif row["CALLSTATE"] == 3:
                liantong += 1
            elif row["CALLSTATE"] == 4:
                duanlian += 1

        speed_max = tempData["SPEED"].max()
        speed_mean = tempData["SPEED"].mean()
        speed_var = tempData["SPEED"].var()
        
        height_max = tempData["HEIGHT"].max()
        height_mean = tempData["HEIGHT"].mean()
        height_var = tempData['HEIGHT'].var()

        sp_he_mean = speed_mean * height_mean

        maxTimelist.append(maxTime)
        maxTime = max(maxTimelist)
        
        total_callstate = len(tempData["CALLSTATE"])
        weizhi_ratio = weizhi / float(total_callstate)
        huchu_ratio = huchu / float(total_callstate)
        huru_ratio = huru / float(total_callstate)
        liantong_ratio = liantong / float(total_callstate)
        duanlian_ratio = duanlian / float(total_callstate)
        
        tempfeature = pd.DataFrame([[p_id,
                                     maxTime,phonerisk,dir_risk,height_risk,
                                     speed_max,speed_mean,speed_var,height_max,height_mean,height_var,sp_he_mean,
                                     zao,wan,shenye,
                                     weizhi_ratio,huchu_ratio,huru_ratio,liantong_ratio,duanlian_ratio
                                    ]],
                                    index=['0'],
                                    columns=columns)
        feature = feature.append(tempfeature,ignore_index=True)

    return feature


In [7]:
def norm_feature(df,feature_name):
    for name in feature_name:
#         df[name] = df[name].map(lambda x:(x-df[name].min())/(df[name].max()-df[name].min()))
        df[name] = pd.DataFrame(MinMaxScaler().fit_transform(pd.DataFrame(df[name])),columns=[name])

# def stan_feature(df,feature_name):
#     for name in feature_name:
#         df[name] = preprocessing.scale(df[name])
#         df[name] = df[name].map(lambda x: preprocessing)

norm_feature_name = ['speed_max','speed_mean','speed_var','height_max','height_var','height_mean','sp_he_mean',
                    ]

In [8]:
def data_process(data):
    data = conver_time(data)
    feature_data = feature_process(data)
    feature_data = feature_data.fillna(method='pad')
    # feature_data = feature_data.dropna(axis=0, how="any")
#     norm_feature(feature_data, norm_feature_name)
    return feature_data

In [9]:
# test_len = len(test)
# train = train.sample(test_len*1.3)
# pre_label = label_process(train[["TERMINALNO","Y"]])

# train = train.drop('Y',axis=1)

In [10]:
feature_train = data_process(train)

In [11]:
feature_train.shape

(100, 20)

In [12]:
feature_train

,p_id,maxTime,phonerisk,dir_risk,height_risk,speed_max,speed_mean,speed_var,height_max,height_mean,height_var,sp_he_mean,zao,wan,shenye,weizhi_ratio,huchu_ratio,huru_ratio,liantong_ratio,duanlian_ratio
0,1.0,3540.0,22.698271,759.505736,253.833059,32.779999,17.489840,138.297543,224.069580,47.848093,907.782777,836.855477,1.0,1.0,1.0,0.489796,0.000000,0.000000,0.003401,0.506803
1,2.0,3180.0,67.556593,3646.161361,376.753980,36.119999,9.287734,153.238930,526.300537,71.400718,10543.556290,663.150842,0.0,1.0,1.0,0.396978,0.000000,0.000000,0.038462,0.564560
2,3.0,3600.0,41.291745,597.444549,840.111283,25.440001,7.987331,37.641736,125.748291,43.942672,295.097428,350.984677,1.0,1.0,0.0,0.870197,0.000000,0.001038,0.000000,0.128764
3,4.0,2700.0,110.310492,659.554444,656.575946,33.310001,6.312753,39.532636,115.885498,31.659250,165.859123,199.857010,1.0,1.0,0.0,0.228535,0.007576,0.003788,0.008838,0.751263
4,5.0,5520.0,39.709307,616.796195,776.837669,53.480000,7.695846,67.545555,117.702576,29.461915,207.698538,226.734356,1.0,1.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000
5,6.0,2700.0,24.592781,571.754381,428.472839,29.570000,11.456653,90.503295,130.427490,9.389893,190.799313,107.576740,1.0,1.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000
6,7.0,2460.0,78.806705,515.198829,627.895442,35.680000,5.705086,61.943701,432.401733,246.410570,3547.505052,1405.793501,1.0,1.0,0.0,0.220922,0.002753,0.001376,0.009635,0.765313
7,8.0,3600.0,28.676366,567.978879,556.496038,42.720001,9.100117,96.098431,188.175537,36.571975,710.369713,332.809257,1.0,1.0,1.0,0.948424,0.000000,0.000000,0.001433,0.050143
8,9.0,5340.0,26.787776,339.889983,290.027412,35.680000,16.274006,136.452488,32.210754,5.575440,70.909271,90.734745,1.0,1.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000
9,10.0,2760.0,223.813791,663.019670,511.592672,32.330002,7.297735,51.864707,400.146973,305.556835,1797.705671,2229.872842,1.0,1.0,1.0,0.004658,0.027950,0.007764,0.099379,0.860248


In [13]:
feature_train.head()

,p_id,maxTime,phonerisk,dir_risk,height_risk,speed_max,speed_mean,speed_var,height_max,height_mean,height_var,sp_he_mean,zao,wan,shenye,weizhi_ratio,huchu_ratio,huru_ratio,liantong_ratio,duanlian_ratio
0,1.0,3540.0,22.698271,759.505736,253.833059,32.779999,17.489840,138.297543,224.069580,47.848093,907.782777,836.855477,1.0,1.0,1.0,0.489796,0.000000,0.000000,0.003401,0.506803
1,2.0,3180.0,67.556593,3646.161361,376.753980,36.119999,9.287734,153.238930,526.300537,71.400718,10543.556290,663.150842,0.0,1.0,1.0,0.396978,0.000000,0.000000,0.038462,0.564560
2,3.0,3600.0,41.291745,597.444549,840.111283,25.440001,7.987331,37.641736,125.748291,43.942672,295.097428,350.984677,1.0,1.0,0.0,0.870197,0.000000,0.001038,0.000000,0.128764
3,4.0,2700.0,110.310492,659.554444,656.575946,33.310001,6.312753,39.532636,115.885498,31.659250,165.859123,199.857010,1.0,1.0,0.0,0.228535,0.007576,0.003788,0.008838,0.751263
4,5.0,5520.0,39.709307,616.796195,776.837669,53.480000,7.695846,67.545555,117.702576,29.461915,207.698538,226.734356,1.0,1.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
feature_train.describe()

,p_id,maxTime,phonerisk,dir_risk,height_risk,speed_max,speed_mean,speed_var,height_max,height_mean,height_var,sp_he_mean,zao,wan,shenye,weizhi_ratio,huchu_ratio,huru_ratio,liantong_ratio,duanlian_ratio
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00,100.000000,100.000000,100.000000,100.000000,100.000000
mean,50.500000,3459.000000,78.495711,627.319222,531.612678,32.413296,10.800129,69.404966,356.060422,188.759950,15029.795568,2219.321047,0.900000,0.910000,0.45,0.634783,0.002828,0.001542,0.015360,0.345486
std,29.011492,1872.215508,174.317182,469.379451,211.447543,6.161771,4.883463,38.813144,579.567785,410.515724,69893.002429,6179.346824,0.301511,0.287623,0.50,0.395661,0.005574,0.005930,0.028534,0.373129
min,1.000000,900.000000,1.580266,214.867458,162.413532,19.330000,4.288839,11.105200,4.000000,2.346456,0.216478,13.559697,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.750000,2160.000000,28.363466,453.219205,382.246475,28.210021,7.320669,38.160376,60.728863,6.437722,78.654125,77.382487,1.000000,1.000000,0.00,0.211501,0.000000,0.000000,0.000000,0.000000
50%,50.500000,3090.000000,42.805348,529.768872,509.024135,32.729999,9.727552,62.357581,111.456997,23.740426,180.247197,226.002061,1.000000,1.000000,0.00,0.842496,0.000000,0.000000,0.000000,0.152999
75%,75.250000,4035.000000,65.691064,682.041173,634.660305,36.212501,13.620368,91.669467,400.833115,119.149538,918.272971,1106.344770,1.000000,1.000000,1.00,1.000000,0.002724,0.001598,0.017564,0.754751
max,100.000000,11820.000000,1685.436788,3646.161361,1177.407773,53.480000,31.048178,178.268100,3411.894775,2827.418959,646680.808695,49563.864817,1.000000,1.000000,1.00,1.000000,0.028986,0.057911,0.157971,0.961224


In [15]:
# feature_train.isnull().sum()

In [16]:
# tempData.isnull().sum()

In [17]:
# tempData[tempData['SPEED_DIFF'].isnull()!=False]

In [18]:
# tempData.head()

In [19]:
# tempData[tempData['SPEED_DIFF']<0].count()